# Data preparation: Stacking the GAF-images to create a 3D input for the CNN
When running this code, 3D arrays for each participant for each epoch are made. Depending on the investigation of contrast, the code is run with appropriate active labels in the event dictionary

#### Import libraries and packages

In [1]:
# Libraries and packages 
import sys
import numba
import numpy as np
from numpy import asarray
import cv2
import glob
import os
from os import mkdir
import pyvista
import pyvistaqt
import pandas as pd
from itertools import compress
from skimage import io

#### Specifying paths and creating participant list

In [ ]:
# Specifying path to participant-folders with epoch-folders containing the GAF images
dir_path = "/path/to/GAF/images" #

# TODO: Specify path to folder where you want to store your .npy files for 1) Epoch_cube_list and 2) Epoch_cube_labels for Participant_i
dir_path_save = "/path/to/folder/for/saving/the/input"

# -- 1) Making a list of participants
participant_list = glob.glob(f"{dir_path}/*/")
if '.DS_Store' in participant_list:
    participant_list.remove('.DS_Store')

print(participant_list)


#### Creatin the event dictionary
Changes are made here in regard to the desired contrast. The followng code show active labels for the Reflection/Verbalization task.

In [ ]:
event_dict_keep_for_stimpres_speak_cont = {
    # -- PERCEPTION

    # Condition_1
                # 'Non-human simple presentation Condition_1 1': 43,
                # 'Non-human simple presentation Condition_1 2': 44,
                # 'Non-human simple presentation Condition_1 3': 45,

                # 'Human simple presentation Condition_1 1': 61,
                # 'Human simple presentation Condition_1 2': 62,
                # 'Human simple presentation Condition_1 3': 63,

    # Condition_2
                # 'Non-human simple presentation Condition_2 1': 52,
                # 'Non-human simple presentation Condition_2 2': 53,
                # 'Non-human simple presentation Condition_2 3': 54,

                # 'Human simple presentation Condition_2 1': 70,
                # 'Human simple presentation Condition_2 2': 71,
                # 'Human simple presentation Condition_2 3': 72,

    # -- REFLECTION --
    
    # Condition_1
                # 'Non-human simple think Condition_1 1': 46, 
                # 'Non-human simple think Condition_1 2': 47,
                # 'Non-human simple think Condition_1 3': 48,

                # 'Human simple think Condition_1 1': 64,             
                # 'Human simple think Condition_1 2': 65,             
                # 'Human simple think Condition_1 3': 66,     
    
    # Condition_2

                'Non-human simple think Condition_2 1': 55, 
                'Non-human simple think Condition_2 2':  56,
                'Non-human simple think Condition_2 3': 57,

                'Human simple think Condition_2 1': 73, 
                'Human simple think Condition_2 2': 74,
                'Human simple think Condition_2 3': 75,

                
    # -- VERBALIZATION --

    # Condition_1
                # 'Non-human simple speak Condition_1 1': 49,
                # 'Non-human simple speak Condition_1 2':  50,
                # 'Non-human simple speak Condition_1 3': 51,

                # 'Human simple speak Condition_1 1': 67,             
                # 'Human simple speak Condition_1 2': 68,             
                # 'Human simple speak Condition_1 3': 69,  
    
    # Condition_2

                'Non-human simple speak Condition_2 1': 58,
                'Non-human simple speak Condition_2 2': 59,
                'Non-human simple speak Condition_2 3': 60,

                'Human simple speak Condition_2 1': 76,
                'Human simple speak Condition_2 2': 77,
                'Human simple speak Condition_2 3': 78

                }

## Convert the 2D GAF images for each epoch into a 3D array and save it as an .npy file along with the corresponding labels


In [ ]:
for participant in participant_list[1:]:
    epoch_cube_list_3D = [] # 3D array 
    epoch_cube_labels_3D = [] # Corresponding label array
    epochfolder_list = glob.glob(f"{participant}/*/")

    for epoch in epochfolder_list:  

        if "_" not in (epoch)[-3:]: # Only include relevant triggers
            if int(epoch[-3:].replace('/','')) in event_dict_keep_for_stimpres_speak_cont.values():
                
                # Create list of labels
                epoch_cube_labels_3D.append(int(epoch[-3:].replace('/',''))) 
                
                os.chdir(epoch) # enter the folder containing the GAF images and retrieve the .png's
                images = glob.glob("*.png")
                images.sort(key = lambda x: int(x.split("-")[0]))

                # Getting the first GAF image as a base for the concatenated image:
                im = cv2.imread(images[0])
                im = np.asarray(im)

                # Give it an extra dimension (needed for modelling)
                im = np.reshape(im, (36,36,1,3)) # 2D (h,w,channel) 3D (h,w,d,ch) 

                # Stack the GAF arrays behind eachother in accordance with spatial channel placement
                for png_file in images[1:]:
                    im_reshaped = cv2.imread(png_file)
                    im_reshaped = np.asarray(im_reshaped)

                    # Give it an extra dimension (needed for modelling)
                    im_reshaped = np.reshape(im_reshaped, (36,36,1,3))
                    
                    # Stack the image behind the others
                    im = np.concatenate((im, im_reshaped), axis = 2)

                #When it's done with the epoch, append im to a list. Should be just as long as the label-list, and should be a list / array of 36,36,3,70 (or 81 normally)
                epoch_cube_list_3D.append(im)

# Saving files as .npy 
    np.save(f"{dir_path_save}/3Darrays_{os.path.basename(os.path.normpath(participant))}_mycontrast.npy", epoch_cube_list_3D, allow_pickle=True)            
    np.save(f"{dir_path_save}/labels_{os.path.basename(os.path.normpath(participant))}_mycontrast.npy", epoch_cube_labels_3D, allow_pickle=True)  
